In [9]:
from Servicios.GoogleDriveService.GoogleDriveService import GoogleDriveService
from Servicios.OrquestadorService.OrchestrateService import OrchestrateService

In [10]:
print("hola")
# Crear instancia de GoogleDriveService
google_drive_service = GoogleDriveService()

# Crear instancia de OrchestrateService, inyectando GoogleDriveService
orchestrate_service = OrchestrateService(google_drive_service)

# Ejecutar el método gdriveservice_listar_archivos desde OrchestrateService
# orchestrate_service.gdriveservice_listar_archivos()

hola
hola2
hola3
hola4
hola5
